#### Monitor Folder and Upload CSV to SQLite

In [1]:
pip install watchdog pandas sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 2.0 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 2.0 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import pandas as pd
import os
from sqlalchemy import create_engine
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [15]:
# 1. Create SQLite database connection
engine = create_engine('sqlite:///uploaded_data.db')

In [19]:
# 2. Define event handler
class CSVHandler(FileSystemEventHandler):
    def on_created(self, event):
        if not event.is_directory and event.src_path.endswith(".csv"):
            print(f"\n📥 New file detected: {event.src_path}")
            try:
                df = pd.read_csv(event.src_path)
                print("📊 File read successfully. First 5 rows:")
                print(df.head())
                df.to_sql("sales_data", con=engine, if_exists="append", index=False)
                print("✅ Data uploaded to database.")
            except Exception as e:
                print(f"❌ Failed to upload {event.src_path}")
                print("❗ Error:", e)


In [20]:
# 3. Start watching folder
folder_to_watch = r"C:\Users\HP\OneDrive\Desktop\Data Science\Jupyter\Navaantrix-DataAnalyst-Tasks\Task3-File Listener and Database Upload\monitored_folder"
eevent_handler = CSVHandler()
observer = Observer()
observer.schedule(eevent_handler, path=folder_to_watch, recursive=False)
observer.start()

print("👀 Monitoring started... Drop a CSV file in the folder.")

👀 Monitoring started... Drop a CSV file in the folder.



📥 New file detected: C:\Users\HP\OneDrive\Desktop\Data Science\Jupyter\Navaantrix-DataAnalyst-Tasks\Task3-File Listener and Database Upload\monitored_folder\Sales_data.csv
📥 New file detected: C:\Users\HP\OneDrive\Desktop\Data Science\Jupyter\Navaantrix-DataAnalyst-Tasks\Task3-File Listener and Database Upload\monitored_folder\Sales_data.csv
📥 New file detected: C:\Users\HP\OneDrive\Desktop\Data Science\Jupyter\Navaantrix-DataAnalyst-Tasks\Task3-File Listener and Database Upload\monitored_folder\Sales_data.csv

📊 File content:
 📊 File read successfully. First 5 rows:
   Transaction ID        Date Product Category             Product Name  \
0           10001  2024-01-01      Electronics            iPhone 14 Pro   
1           10002  2024-01-02  Home Appliances         Dyson V11 Vacuum   
2           10003  2024-01-03         Clothing         Levi's 501 Jeans   
3           10004  2024-01-04            Books        The Da Vinci Code   
4           10005  2024-01-05  Beauty Products  Ne

In [21]:
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()

In [22]:
import sqlite3

conn = sqlite3.connect("uploaded_data.db")
df_check = pd.read_sql_query("SELECT * FROM sales_data", conn)
df_check

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue,Region,Payment Method
0,10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card
1,10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal
2,10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card
3,10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card
4,10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal
...,...,...,...,...,...,...,...,...,...
715,10236,2024-08-23,Home Appliances,Nespresso Vertuo Next Coffee and Espresso Maker,1,159.99,159.99,Europe,PayPal
716,10237,2024-08-24,Clothing,Nike Air Force 1 Sneakers,3,90.00,270.00,Asia,Debit Card
717,10238,2024-08-25,Books,The Handmaid's Tale by Margaret Atwood,3,10.99,32.97,North America,Credit Card
718,10239,2024-08-26,Beauty Products,Sunday Riley Luna Sleeping Night Oil,1,55.00,55.00,Europe,PayPal
